In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [2]:
# Loading Data
df1 = pd.read_excel(r'C:\Users\Hendelle.DESKTOP-V5JNA20\Downloads\02 - 2023.xlsx')

In [3]:
df1 = df1[df1['RETAILQUANTITY(UOM)']>0].reset_index(drop=True) # remove negative transaction
df1 = df1[df1['SALESUOM']!='AU'].reset_index(drop=True) # remove UOM AU
#df = df[df['SALESUOM']!='PAC'].reset_index(drop=True) # remove UOM PAC
df1 = df1[df1['SALESUOM']!='UNDEFINED'].reset_index(drop=True)
df1 = df1.drop('BRANDNAME',axis = 1)
df1.head()

,RETAILSTOREID,SITENAME,STORENAME,BUSINESSDAYDATE,MONTHNUM,YEARNUM,TRANSNUMBER,POSTERMINAL,TRANSID,CATEGORY,...,VENDORNAME,POSTINGGROUP,MATERIALNUMBER,ITEMDESCRIPTION,SALESUOM,ITEM,RETAILQUANTITY(UOM),GROSSAMT,DISCOUNTAMT,SALESAMT
0,B101,Gapan,B101-Gapan,2023-02-20,2,2023,7683,POS02,B101POS027683,DIY Finishing,...,Allgemeine-Bau-Chemie Phil. Inc.,OUTRIGHT,10000971,ABC REDIFIX 4L,PC,10000971-ABC REDIFIX 4L,1.0,930.0,0.0,930.0
1,B101,Gapan,B101-Gapan,2023-02-08,2,2023,18372,POS01,B101POS0118372,DIY Finishing,...,Allgemeine-Bau-Chemie Phil. Inc.,OUTRIGHT,10000975,ABC-MULTIBOND(VINYL/PARQUET AD,PC,10000975-ABC-MULTIBOND(VINYL/PARQUET AD,1.0,706.0,0.0,706.0
2,B101,Gapan,B101-Gapan,2023-02-22,2,2023,7693,POS02,B101POS027693,DIY Finishing,...,Allgemeine-Bau-Chemie Phil. Inc.,OUTRIGHT,10001025,ABC-TILE GROUT F12 BLACK 2KG/,PC,10001025-ABC-TILE GROUT F12 BLACK 2KG/,1.0,90.0,0.0,90.0
3,B101,Gapan,B101-Gapan,2023-02-12,2,2023,18449,POS01,B101POS0118449,DIY Finishing,...,Allgemeine-Bau-Chemie Phil. Inc.,OUTRIGHT,10001031,ABC-TILE GROUT F16 CREAM 2KG/,PC,10001031-ABC-TILE GROUT F16 CREAM 2KG/,1.0,90.0,0.0,90.0
4,B101,Gapan,B101-Gapan,2023-02-13,2,2023,18467,POS01,B101POS0118467,DIY Finishing,...,Allgemeine-Bau-Chemie Phil. Inc.,OUTRIGHT,10001031,ABC-TILE GROUT F16 CREAM 2KG/,PC,10001031-ABC-TILE GROUT F16 CREAM 2KG/,1.0,90.0,0.0,90.0


In [4]:
#Checking of missing values
missing_values_count = df1.isnull().sum()
missing_values_count

RETAILSTOREID          0
SITENAME               0
STORENAME              0
BUSINESSDAYDATE        0
MONTHNUM               0
YEARNUM                0
TRANSNUMBER            0
POSTERMINAL            0
TRANSID                0
CATEGORY               0
SUBCATEGORY            0
CLASS                  0
SUBCLASS               0
VENDORCODE             0
VENDORNAME             0
POSTINGGROUP           0
MATERIALNUMBER         0
ITEMDESCRIPTION        0
SALESUOM               0
ITEM                   0
RETAILQUANTITY(UOM)    0
GROSSAMT               0
DISCOUNTAMT            0
SALESAMT               0
dtype: int64

In [5]:
#removing white spaces
df1['ITEM'] = df1['ITEM'].str.strip() 
df1['ITEMDESCRIPTION'] = df1['ITEMDESCRIPTION'].str.strip()

# Adding new column: Unique ID
df1['Unique ID']=df1['YEARNUM'].astype(str) + df1['MONTHNUM'].astype(str)+ df1['TRANSNUMBER'].astype(str) + df1['TRANSID'].astype(str) 

#converting invoice number to be string
df1['Unique ID'] = df1['Unique ID'].astype('str')

df1.reset_index(drop=True,inplace=True)
df1.head()

,RETAILSTOREID,SITENAME,STORENAME,BUSINESSDAYDATE,MONTHNUM,YEARNUM,TRANSNUMBER,POSTERMINAL,TRANSID,CATEGORY,...,POSTINGGROUP,MATERIALNUMBER,ITEMDESCRIPTION,SALESUOM,ITEM,RETAILQUANTITY(UOM),GROSSAMT,DISCOUNTAMT,SALESAMT,Unique ID
0,B101,Gapan,B101-Gapan,2023-02-20,2,2023,7683,POS02,B101POS027683,DIY Finishing,...,OUTRIGHT,10000971,ABC REDIFIX 4L,PC,10000971-ABC REDIFIX 4L,1.0,930.0,0.0,930.0,202327683B101POS027683
1,B101,Gapan,B101-Gapan,2023-02-08,2,2023,18372,POS01,B101POS0118372,DIY Finishing,...,OUTRIGHT,10000975,ABC-MULTIBOND(VINYL/PARQUET AD,PC,10000975-ABC-MULTIBOND(VINYL/PARQUET AD,1.0,706.0,0.0,706.0,2023218372B101POS0118372
2,B101,Gapan,B101-Gapan,2023-02-22,2,2023,7693,POS02,B101POS027693,DIY Finishing,...,OUTRIGHT,10001025,ABC-TILE GROUT F12 BLACK 2KG/,PC,10001025-ABC-TILE GROUT F12 BLACK 2KG/,1.0,90.0,0.0,90.0,202327693B101POS027693
3,B101,Gapan,B101-Gapan,2023-02-12,2,2023,18449,POS01,B101POS0118449,DIY Finishing,...,OUTRIGHT,10001031,ABC-TILE GROUT F16 CREAM 2KG/,PC,10001031-ABC-TILE GROUT F16 CREAM 2KG/,1.0,90.0,0.0,90.0,2023218449B101POS0118449
4,B101,Gapan,B101-Gapan,2023-02-13,2,2023,18467,POS01,B101POS0118467,DIY Finishing,...,OUTRIGHT,10001031,ABC-TILE GROUT F16 CREAM 2KG/,PC,10001031-ABC-TILE GROUT F16 CREAM 2KG/,1.0,90.0,0.0,90.0,2023218467B101POS0118467


In [6]:
#separating transactions
basket = df1.groupby(['Unique ID', 'SUBCLASS'])['RETAILQUANTITY(UOM)'].sum().unstack()
basket 

SUBCLASS,"""Back Packs, Sling B","""Tv Rack, Glass""","""Tv Rack,Wooden""",(blocked) Duplo,1 Door (Mjr Apps),1 Pc Fitted Bedsheet,1 Pc Pillowcase,1 Seater,1 Seater Power Recli,1 Seater Recliners,...,Wooden Spice Rack,Wooden Tea Box,Wooden Utensil Set,Wooden Utensils,Wrappers,Writing Accessories,Writing Stationery,Wrought Iron,Xmas Decrtve TbleTop,Yoga Accessories
Unique ID,,,,,,,,,,,,,,,,,,,,,
2023210000HS27POS0110000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023210001HS27POS0110001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023210002HS27POS0110002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023210003HS27POS0110003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20232100041XHS06DPOS8X100041X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202329997HS27POS019997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
202329998HS27POS019998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
202329999HS27POS019999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
#converting all positive values to 1 and everything else to 0
def my_encode_units(x):
    if x >= 0: return 1
    elif x < 0: return 0
    else: return 0
    
basket_set = basket.applymap(my_encode_units)
basket_set

SUBCLASS,"""Back Packs, Sling B","""Tv Rack, Glass""","""Tv Rack,Wooden""",(blocked) Duplo,1 Door (Mjr Apps),1 Pc Fitted Bedsheet,1 Pc Pillowcase,1 Seater,1 Seater Power Recli,1 Seater Recliners,...,Wooden Spice Rack,Wooden Tea Box,Wooden Utensil Set,Wooden Utensils,Wrappers,Writing Accessories,Writing Stationery,Wrought Iron,Xmas Decrtve TbleTop,Yoga Accessories
Unique ID,,,,,,,,,,,,,,,,,,,,,
2023210000HS27POS0110000,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2023210001HS27POS0110001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2023210002HS27POS0110002,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2023210003HS27POS0110003,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20232100041XHS06DPOS8X100041X,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202329997HS27POS019997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
202329998HS27POS019998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
202329999HS27POS019999,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
#Generating frequent itemsets
freq_items = apriori(basket_set, min_support=0.01, use_colnames=True, max_len=3)
freq_items.sort_values(by = "support", ascending = False)
#freq_items[freq_items['itemsets'].str.contains('Fragrance Oil',regex = False)]

C:\Users\Hendelle.DESKTOP-V5JNA20\anaconda3\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
21,0.079262,(Oth ElectricalSupply)
27,0.060816,(Oth. Bonding Agents)
28,0.058083,(Oth. Bulbs)
23,0.056656,(Oth Paints & Stains)
25,0.054336,(Oth Sink&ToiletRpair)
31,0.043361,(Oth. Faucets)
18,0.040455,(Oth Cleaning Imp.)
32,0.039776,(Oth. Hand Tools)
35,0.039579,(Oth. Pntng Ad & Gdgt)
38,0.032885,(Othr Home Essentials)


In [9]:
#Generating rules
rules = association_rules(freq_items, metric='confidence',min_threshold=0.05)
rules.sort_values(by = "support", ascending = False)
#rules[rules['consequents'].str.contains('Fragrance Oil', regex=False)]


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2,(Oth Paints & Stains),(Oth. Pntng Ad & Gdgt),0.056656,0.039579,0.020343,0.359066,9.072172,0.018101,1.498472
3,(Oth. Pntng Ad & Gdgt),(Oth Paints & Stains),0.039579,0.056656,0.020343,0.513997,9.072172,0.018101,1.941023
4,(Oth Sink&ToiletRpair),(Oth. Faucets),0.054336,0.043361,0.016590,0.305327,7.041519,0.014234,1.377107
5,(Oth. Faucets),(Oth Sink&ToiletRpair),0.043361,0.054336,0.016590,0.382609,7.041519,0.014234,1.531709
0,(Oth. Bulbs),(Oth ElectricalSupply),0.058083,0.079262,0.012454,0.214421,2.705217,0.007851,1.172050
1,(Oth ElectricalSupply),(Oth. Bulbs),0.079262,0.058083,0.012454,0.157128,2.705217,0.007851,1.117509


In [10]:
#Filtering rules based on condition
filteredrules = rules[(rules['lift'] >=2) & (rules['confidence'] >= 0.3)]
filteredrules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2,(Oth Paints & Stains),(Oth. Pntng Ad & Gdgt),0.056656,0.039579,0.020343,0.359066,9.072172,0.018101,1.498472
3,(Oth. Pntng Ad & Gdgt),(Oth Paints & Stains),0.039579,0.056656,0.020343,0.513997,9.072172,0.018101,1.941023
4,(Oth Sink&ToiletRpair),(Oth. Faucets),0.054336,0.043361,0.016590,0.305327,7.041519,0.014234,1.377107
5,(Oth. Faucets),(Oth Sink&ToiletRpair),0.043361,0.054336,0.016590,0.382609,7.041519,0.014234,1.531709
